In [2]:
import pandas as pd

YEARS = [2018, 2019]

data = pd.DataFrame()

#ingest
for year in YEARS:
    data = data.append(pd.read_csv(f"data/play_by_play_cleaned_{year}.csv.gz", compression='gzip', low_memory=False).reset_index(drop=True), sort=True)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 300)
data.reset_index(drop=True, inplace=True)

In [3]:
DOWN_UPPER = 3
NEUTRAL = True
if NEUTRAL:
    neutral = data.loc[ (data.play_type!="no_play") & (data.qtr<5) &
                           (data.down<=DOWN_UPPER) & (data.half_seconds_remaining>120) &
                                     (data.wp>=.2) & (data.wp<=.8)].reset_index(drop=True)
else:
     neutral = data.loc[ (data.play_type!="no_play") & (data.qtr<5)].reset_index(drop=True)

In [5]:
import numpy as np
def epa_success(epa):
    return 1 if epa>0 else 0

runs = neutral.loc[(data.play_type=='run')]
passes = neutral.loc[(data.play_type=='pass')]
sea_runs = runs.loc[(data.posteam=='SEA')]
sea_passes = passes.loc[(data.posteam=='SEA')]

run_aggs = runs.groupby(['season', 'qtr']).agg(
                    play_count = ('play_id', 'count'),
                    avg_epa = ('epa', 'mean'),
                    epa_succ_rate = ('epa', lambda x: (x.map(epa_success).mean()))).round(2)
pass_aggs = passes.groupby(['season', 'qtr']).agg(
                    play_count = ('play_id', 'count'),
                    avg_epa = ('epa', 'mean'),
                    epa_succ_rate = ('epa', lambda x: (x.map(epa_success).mean()))).round(2)
sea_pass_aggs = sea_passes.groupby(['season', 'qtr']).agg(
                    play_count = ('play_id', 'count'),
                    avg_epa = ('epa', 'mean'),
                    epa_succ_rate = ('epa', lambda x: (x.map(epa_success).mean()))).round(2)
sea_run_aggs = sea_runs.groupby(['season', 'qtr']).agg(
                    play_count = ('play_id', 'count'),
                    avg_epa = ('epa', 'mean'),
                    epa_succ_rate = ('epa', lambda x: (x.map(epa_success).mean()))).round(2)
all = (run_aggs.join(pass_aggs, lsuffix='_run', rsuffix='_pass')).join(sea_run_aggs.join(sea_pass_aggs, lsuffix='_run', rsuffix='_pass'), lsuffix='_lg', rsuffix='_sea') 
all.reset_index(inplace=True)
all.loc[all.season==2019]
# all_run = run_aggs.join(sea_run_aggs, lsuffix='_lg', rsuffix='_sea')
# all_pass = pass_aggs.join(sea_pass_aggs, lsuffix='_lg', rsuffix='_sea')

,season,qtr,play_count_run_lg,avg_epa_run_lg,epa_succ_rate_run_lg,play_count_pass_lg,avg_epa_pass_lg,epa_succ_rate_pass_lg,play_count_run_sea,avg_epa_run_sea,epa_succ_rate_run_sea,play_count_pass_sea,avg_epa_pass_sea,epa_succ_rate_pass_sea
4,2019,1,2587,0.01,0.43,4164,-0.02,0.43,134,-0.05,0.44,135,-0.12,0.42
5,2019,2,1790,-0.01,0.42,2830,0.02,0.44,105,-0.04,0.41,102,0.26,0.50
6,2019,3,1305,0.02,0.45,2251,0.01,0.45,60,0.02,0.48,78,0.12,0.50
7,2019,4,852,0.04,0.44,1259,-0.01,0.42,15,0.58,0.60,24,-0.23,0.29
